# 전임교원 수 및 1인당 학생 수 전처리 노트북

모델링에 사용할 수 있는 형태로 변환하는 과정을 포함합니다.
- 기준년도: 2023년에 존재하는 학교만 유지
- 사이버대학 제거
- 전임교원수는 정원 기준으로 통합
- 전임교원수 및 1인당 학생 수가 모두 0인 행 제거

In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# 예시: 2023년 파일에서 유효한 학교 추출
df_2023 = pd.read_excel("2023년 _대학_6-나-(1). 전임교원 1인당 학생 수 및 전임교원 확보율_학교별자료.xlsx", header=None)
valid_schools = df_2023[5].dropna().unique().tolist()
valid_schools = [s for s in valid_schools if "사이버" not in s]

In [ ]:
def process_faculty_file(path, year, school_col, faculty_col, per_std_col, per_enr_col):
    df = pd.read_excel(path, header=None)
    df = df.iloc[6:, [school_col, faculty_col, per_std_col, per_enr_col]].copy()
    df.columns = ['학교', '전임교원수', '1인당 학생수(정원기준)', '1인당 학생수(재학생기준)']

    for col in ['전임교원수', '1인당 학생수(정원기준)', '1인당 학생수(재학생기준)']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df[df['학교'].notna()]
    df = df[~df['학교'].str.contains('사이버')]
    df = df[df['학교'].isin(valid_schools)]
    df = df[~((df['전임교원수'] == 0) & (df['1인당 학생수(정원기준)'] == 0) & (df['1인당 학생수(재학생기준)'] == 0))]
    df["기준년도"] = year
    return df

In [ ]:
# 예시 연도별 구성 및 통합
file_info = {
    2023: ("2023년 ~.xlsx", 5, 12, 13, 14),
    2022: ("2022년 ~.xlsx", 5, 12, 13, 14),
    2021: ("2021년 ~.xlsx", 5, 12, 13, 14),
    # 나머지 연도들도 같은 방식으로 추가
}

all_data = []
for year, (filename, scol, fcol, pscol, pecol) in file_info.items():
    df = process_faculty_file(filename, year, scol, fcol, pscol, pecol)
    all_data.append(df)

merged = pd.concat(all_data, ignore_index=True)
merged = merged[['기준년도', '학교', '전임교원수', '1인당 학생수(정원기준)', '1인당 학생수(재학생기준)']]
merged.to_csv("전임교원_1인당학생수_2010_2023_통합.csv", index=False, encoding="cp949")
merged.head()